# Target Propagation - Tutorial 1
## Standard Target Propagation

A simple implementation of target propagation to confirm that it works.

In [ ]:
import initialize

In [ ]:
from torchvision.datasets import FashionMNIST
from torchvision import transforms

from tools.training import train, classify
from tools.learners.target_prop import TargetPropLearner
from tools.learners.target_prop import BaselineLearner1

# Steps

1) Create each layer (AutoencoderLearner)
2) Create the TargetPropLearner
3) Run the training on the baseline
4) Run the training on the target propagation learner 

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

training_dataset = FashionMNIST(
    '../../Datasets/',
    transform=transform, download=True
)

testing_dataset = FashionMNIST(
    '../../Datasets/', train=False,
    transform=transform, download=True
)



In [ ]:
learner = BaselineLearner1(
    784, 300, 300, 300, 10
)
train(learner, training_dataset, 10, device='cpu')
classify(learner, testing_dataset)

# TargetPropLearner

In [ ]:
learner = TargetPropLearner(
    784, 300, 300, 300, 10, 
)


train(learner, training_dataset, 50, device='cpu')

classify(learner, testing_dataset)